In [1]:
from tqdm.notebook import tqdm
import pandas as pd
from typing import Optional, List, Tuple
from datasets import Dataset
import matplotlib.pyplot as plt

pd.set_option("display.max_colwidth", None)  # This will be helpful when visualizing retriever outputs

In [2]:
from langchain.document_loaders import PyMuPDFLoader
import os
import yake
"""
def load_pdfs_from_folder(folder_path):
    pdf_files = [f for f in os.listdir(folder_path) if f.endswith(".pdf")]
    documents = []
    for pdf_file in tqdm(pdf_files):
        loader = PyMuPDFLoader(REMOVED_SECRET(folder_path, pdf_file))
        documents.extend(loader.load())
    return documents
"""
def load_pdfs_from_folder(folder_path):
    """
    This function loads PDFs from a chosen folder - or a 'local database'. 
    For each file detected, it checks if the file is empty. 
    The file is then loaded into PyMuPDF. 
    The document is added to a list called 'documents'. 
    And then at the end, that list is returned. 

    I am planning to add functionality (perhaps another function) that will store keywords from each PDF.
    """
    pdf_files = [f for f in os.listdir(folder_path) if f.endswith(".pdf")]
    documents = []
    for pdf_file in tqdm(pdf_files):
        pdf_path = REMOVED_SECRET(folder_path, pdf_file)
        try:
            #Check if the file is empty before loading
            if REMOVED_SECRET(pdf_path) == 0:
                print(f"Skipping empty file: {pdf_file}")
                continue  #Move on to the next file

            loader = PyMuPDFLoader(pdf_path)
            documents.extend(loader.load())
        except Exception as e:  #Catch potential errors during loading
            print(f"Error loading file {pdf_file}: {e}")
    return documents

In [3]:
pdf_folder_path = "local_database"
pdf_data = load_pdfs_from_folder(pdf_folder_path)

  0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
from REMOVED_SECRET import Document as LangchainDocument

RAW_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=doc.page_content, metadata={"source": doc.metadata["source"]}) for doc in tqdm(pdf_data)
]


  0%|          | 0/221 [00:00<?, ?it/s]

In [ ]:
kw_extractor = yake.KeywordExtractor()
keywords = {}


def get_keywords_from_file(documents):
    for 

In [5]:
from tqdm.notebook import tqdm
import pandas as pd
from typing import Optional, List, Tuple
from datasets import Dataset
import matplotlib.pyplot as plt
import yake  # Keyword extractor
from langchain.document_loaders import PyMuPDFLoader
from REMOVED_SECRET import Document as LangchainDocument
import os


pd.set_option("display.max_colwidth", None)


def load_pdfs_from_folder(folder_path):
    """
    Loads PDFs from a folder, skipping empty files.
    Adds keyword extraction to each document.
    """
    pdf_files = [f for f in os.listdir(folder_path) if f.endswith(".pdf")]
    documents = []
    keyword_dict = {}  # Dictionary to store keywords by document ID

    for pdf_file in tqdm(pdf_files):
        pdf_path = REMOVED_SECRET(folder_path, pdf_file)
        try:
            if REMOVED_SECRET(pdf_path) == 0:
                print(f"Skipping empty file: {pdf_file}")
                continue

            loader = PyMuPDFLoader(pdf_path)
            loaded_docs = loader.load()

            for doc in loaded_docs:  # Extract keywords for each loaded page
                # Keyword extraction (you can adjust parameters)
                kw_extractor = yake.KeywordExtractor(lan="en", n=3, dedupLim=0.9, top=10)
                keywords = kw_extractor.extract_keywords(doc.page_content)
                keywords = [kw[0] for kw in keywords]  # Get just the keyword strings

                # Create LangchainDocument with keywords added
                langchain_doc = LangchainDocument(
                    page_content=doc.page_content, 
                    metadata={"source": doc.metadata["source"], "keywords": keywords}
                )
                documents.append(langchain_doc)

                # Add to keyword dictionary
                doc_id = langchain_doc.metadata["source"]  # Unique ID (likely the file path)
                keyword_dict[doc_id] = keywords 

        except Exception as e:
            print(f"Error loading file {pdf_file}: {e}")

    return documents, keyword_dict


# Load PDFs and get keywords
pdf_folder_path = "local_database"
pdf_data, keyword_dict = load_pdfs_from_folder(pdf_folder_path)

# Your existing LangchainDocument creation, now with keywords
RAW_KNOWLEDGE_BASE = [doc for doc in tqdm(pdf_data)] 

print("Sample keyword entries:", list(keyword_dict.items())[:3])  # See some results


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/221 [00:00<?, ?it/s]

Sample keyword entries: [('local_REMOVED_SECRETde20231-133446952049283794.pdf', ['Birmingham City University', 'Birmingham City', 'City University', 'Lumivero', 'Birmingham', 'University', 'Assassin Creed', 'City', 'August', 'Candy Crush Saga']), ('local_database/How-it-Works-booklet.pdf', ['Logo', 'Parliament']), ('local_database/c4611_sample_explain.pdf', ['PDF Bookmark Sample', 'Bookmark Sample Page', 'PDF Bookmark', 'Sample Page', 'Bookmark Sample', 'Place', 'Output Designer', 'PDF', 'Page', 'Bookmark'])]
